# Trabalho de conclusão do curso BI Master

Aluno: Cristiano Leonardo Conceição

## Turismo Inteligente RJ: Uma Abordagem Integrada com RAG e Otimização de Planejamento

## Resumo

Este trabalho propõe uma solução inovadora para turistas que visitam a cidade do Rio de Janeiro, oferecendo um maior conhecimento sobre os principais pontos turísticos e as melhores rotas para seus passeios. A primeira etapa do projeto consiste no desenvolvimento de um modelo baseado em Retrieval-Augmented Generation (RAG), que fornece informações detalhadas e personalizadas sobre os locais de interesse. Na segunda etapa, será realizada a modelagem de uma solução de otimização de planejamento, permitindo que, após a seleção dos pontos turísticos, sejam traçadas as rotas mais eficientes. Essa abordagem visa ajudar os turistas a otimizar seu tempo e reduzir os custos com deslocamento, proporcionando uma experiência mais prática e agradável.


## RAG

Retrieval-Augmented Generation (RAG) é o processo de otimizar a saída de um grande modelo de linguagem, de forma que ele faça referência a uma base de conhecimento confiável fora das suas fontes de dados de treinamento antes de gerar uma resposta. Grandes modelos de linguagem (LLMs) são treinados em grandes volumes de dados e usam bilhões de parâmetros para gerar resultados originais para tarefas como responder a perguntas, traduzir idiomas e concluir frases. A RAG estende os já poderosos recursos dos LLMs para domínios específicos ou para a base de conhecimento interna de uma organização sem a necessidade de treinar novamente o modelo. É uma abordagem econômica para melhorar a produção do LLM, de forma que ele permaneça relevante, preciso e útil em vários contextos.

## Instalação das bibliotecas
pymilvus - Banco de dados vetorial (https://milvus.io/intro)<br/>
wikipedia-api - Api do wikipedia. (https://pypi.org/project/Wikipedia-API/)

In [ ]:
!pip install -U pymilvus
!pip install -U "pymilvus[model]"
!pip install -U wikipedia-api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.1/201.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.7.1-py3-none-any.whl size=14346 sha256=01fe5827ee1051e050183756c4d6907a9584072067c0233a256bfcc76b7d9da7
  Stored in directory: /root/.cache/pip/wheels/4c/96/18/b9201cc3e8b47b02b510460210cfd832ccf10c0c4dd0522962
Successfully built wikipedia-api


## Importar bibliotecas

In [ ]:
import wikipediaapi
import pandas as pd
from pymilvus import (
    MilvusClient,
    DataType
)
from pymilvus import model


## Criação do banco de dados vetorial Milvus.

Antes de criar uma coleção no Milvus, pode usar estas funções para gerar embeddings para os seus conjuntos de dados, simplificando o processo de preparação de dados e pesquisas vectoriais. <br/>
Nossa coleção de dados terá o seguinte Schema:<br/>
ID = Primary Key, identidade do registro da coleção.<br/>
Vector = campo vetorial multi-dimensional que registra cada Embedding da coleção.<br/>
Text = Texto inserido na coleção.<br/>
Subject = Assunto do do texto que foi inserido.<br/><br/>
Vamos criar um índice no campo Vector com o Tipo L2 para otimizar a pesquisa que será feita com busca por distância euclidiana.



In [ ]:
client = MilvusClient("tur_rj.db")

if client.has_collection(collection_name="tur_rj_collection"):
    client.drop_collection(collection_name="tur_rj_collection")

schema = client.create_schema(enable_dynamic_field=True, auto_id=True)
schema.add_field("id", DataType.INT64, is_primary=True)
schema.add_field("vector", DataType.FLOAT_VECTOR, dim=768)

index_params = client.prepare_index_params()
index_params.add_index(field_name = "vector", metric_type="L2")

client.create_collection(
    collection_name="tur_rj_collection",
    schema=schema,
    dimension=768,
    index_params=index_params,
    consistency_level="Strong",  # Strong consistency level
)

embedding_fn = model.DefaultEmbeddingFunction()

DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: d0ec02ce8e6f43ac82fc6bb6a7b93ecf
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created collection: tur_rj_collection
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created an index on collection: tur_rj_collection
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/827 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/245 [00:00<?, ?B/s]

model.onnx:   0%|          | 0.00/46.9M [00:00<?, ?B/s]

## wikipediaapi e inserção das informações no Milvus

Para pegar as informações dos pontos turísticos vamos usar a API do wikipedia.

Faremos um split para cada frase que termina em "." ponto, que serão os nossos registros na coleção criada no Milvus.<br/>
Cada pedaço de texto (chunk) será transformado com uma função de embedding no campo Vector e assim podemos fazer a busca por similaridade de vetores.

In [ ]:
wiki_wiki = wikipediaapi.Wikipedia('BI Master (cristianoconce@gmail.com)', 'pt')

lugares = ['Pão de açucar', 'Cristo Redentor', 'Lagoa Rodrigo de Freitas', 'Praia de Copacabana', 'Praia de Ipanema', 'Arpoador', 'Praça Mauá', 'Praia da Barra da Tijuca']

for lugar in lugares:
  page_py = wiki_wiki.page(lugar)
  txt = page_py.text
  docs = txt.split(". ")
  vectors = embedding_fn.encode_documents(docs)
  data = [
      {"vector": vectors[i], "text": docs[i], "subject": lugar}
      for i in range(len(vectors))
  ]
  client.insert(collection_name="tur_rj_collection", data=data)




## Amostra de Dados
Mostramos os 5 primeiros registros que foram inseridos na coleção do Milvus.

In [ ]:
res = client.query(
    collection_name="tur_rj_collection",
    output_fields=["text", "subject", "vector"],
    limit=5,
)

df = pd.DataFrame(res)
df

,id,text,subject,vector
0,454065019754381312,Monumento Natural dos Morros do Pão de Açúcar ...,Pão de açucar,"[0.028441926, 0.0567731, 0.030196112, 0.007155..."
1,454065019754381313,É composto pelo Morro do Pão de Açúcar (que dá...,Pão de açucar,"[0.023266044, 0.08579364, 0.03239781, -0.03854..."
2,454065019754381314,"Junto com a estátua do Cristo Redentor, é o ma...",Pão de açucar,"[0.030441746, 0.033030402, 0.037122056, -0.014..."
3,454065019754381315,"Pelas características únicas, margeado pelas á...",Pão de açucar,"[0.021702724, 0.06714817, 0.0047133598, -0.048..."
4,454065019754381316,"Conhecido como Bondinho do Pão de Açúcar, o te...",Pão de açucar,"[0.015595551, 0.026514748, -0.011915928, -0.00..."


##Análise de dados

In [ ]:
class Resposta:
  def __init__(self, id, distance, subject, text):
    self.id = id
    self.distance = distance
    self.subject = subject
    self.text = text


## Pesquisa vetorial com métricas

Os tipos de métricas de semelhança são utilizados para medir as semelhanças entre vectores. Atualmente, o Milvus suporta a distância euclidiana (L2), o produto interno (IP), a semelhança cosseno (COSINE) e os tipos de métricas binárias. Pode escolher o tipo de métrica mais adequado com base no seu cenário. Para obter mais informações, consulte Métricas de similaridade. Usamos a distância euclidiana (L2) nesse projeto, para busca de informações em texto. Também podemos fazer uma pesquisa híbrida, usando o parâmetro de filter. No nosso caso vamos filtrar por assunto. <br/>
Para cada pesquisa teremos o top 3 resultados das melhores respostas da busca por similaridade vetorial, podemos verificar no campo distance os melhores resultados.

### Pesquisa Pão de açucar

In [ ]:
res = client.search(
    collection_name="tur_rj_collection",
    data=embedding_fn.encode_queries(["características do relevo"]), #
    output_fields=["text", "subject"],
    limit=3,  # Return top 3 results
    filter='subject like "%Pão de açucar%"'
)

busca = []
respostas = []

for item in res[0]:
  respostas.append(item)

for resp in respostas:
    busca.append(Resposta(resp['id'],
                          resp['distance'],
                          resp['entity']['subject'],
                          resp['entity']['text']).__dict__)

df = pd.DataFrame(busca)
df

,id,distance,subject,text
0,454065019754381315,1.045817,Pão de açucar,"Pelas características únicas, margeado pelas á..."
1,454065019754381326,1.058820,Pão de açucar,O monumento natural faz parte do Mosaico Cario...
2,454065019754381357,1.246612,Pão de açucar,O resgate foi feito com o auxílio do helicópte...


### Pesquisa Cristo Redentor

In [ ]:
res = client.search(
    collection_name="tur_rj_collection",
    data=embedding_fn.encode_queries(["mundialmente"]), #
    output_fields=["text", "subject"],
    limit=3,  # Return top 3 results
    filter='subject like "%Cristo Redentor%"'
)

busca = []
respostas = []

for item in res[0]:
  respostas.append(item)

for resp in respostas:
    busca.append(Resposta(resp['id'],
                          resp['distance'],
                          resp['entity']['subject'],
                          resp['entity']['text']).__dict__)

df = pd.DataFrame(busca)
df

,id,distance,subject,text
0,454065033109831780,1.049471,Cristo Redentor,O monumento foi construído na França a partir ...
1,454065033109831791,1.083006,Cristo Redentor,A estátua do Cristo Redentor de braços abertos...
2,454065033109831794,1.144009,Cristo Redentor,O rosto da estátua foi criado pelo escultor Gh...


### Pesquisa Lagoa Rodrigo de Freitas

In [ ]:
res = client.search(
    collection_name="tur_rj_collection",
    data=embedding_fn.encode_queries(["nível de vida populacional"]), #
    output_fields=["text", "subject"],
    limit=3,  # Return top 3 results
    filter='subject like "%Lagoa Rodrigo de Freitas%"'
)

busca = []
respostas = []

for item in res[0]:
  respostas.append(item)

for resp in respostas:
    busca.append(Resposta(resp['id'],
                          resp['distance'],
                          resp['entity']['subject'],
                          resp['entity']['text']).__dict__)

df = pd.DataFrame(busca)
df

,id,distance,subject,text
0,454065039152513253,0.807265,Lagoa Rodrigo de Freitas,"É uma região de classes média-alta e alta, com..."
1,454065039152513254,0.810663,Lagoa Rodrigo de Freitas,"Desde 1995, na época de Natal, há a tradição d..."
2,454065039152513250,0.846969,Lagoa Rodrigo de Freitas,O canal dragado passou a denominar-se Canal do...


### Pesquisa Copacabana

In [ ]:
res = client.search(
    collection_name="tur_rj_collection",
    data=embedding_fn.encode_queries(["em termos populacionais"]), #
    output_fields=["text", "subject"],
    limit=3,  # Return top 3 results
    filter='subject like "%Copacabana%"'
)

busca = []
respostas = []

for item in res[0]:
  respostas.append(item)

for resp in respostas:
    busca.append(Resposta(resp['id'],
                          resp['distance'],
                          resp['entity']['subject'],
                          resp['entity']['text']).__dict__)

df = pd.DataFrame(busca)
df

,id,distance,subject,text
0,454065041890607382,0.791623,Praia de Copacabana,Em razão da projeção internacional como cartão...
1,454065041890607380,0.918010,Praia de Copacabana,"Limitada pela Avenida Atlântica, Copacabana é ..."
2,454065041890607379,0.920244,Praia de Copacabana,É considerada uma das praias mais famosas do m...


### Pesquisa Ipanema

In [ ]:
res = client.search(
    collection_name="tur_rj_collection",
    data=embedding_fn.encode_queries(["restaurantes"]), #
    output_fields=["text", "subject"],
    limit=3,  # Return top 3 results
    filter='subject like "%Ipanema%"'
)

busca = []
respostas = []

for item in res[0]:
  respostas.append(item)

for resp in respostas:
    busca.append(Resposta(resp['id'],
                          resp['distance'],
                          resp['entity']['subject'],
                          resp['entity']['text']).__dict__)

df = pd.DataFrame(busca)
df

,id,distance,subject,text
0,454065050363887948,0.863059,Praia de Ipanema,\nMuitos desses locais estão estrategicamente ...
1,454065050363887947,1.027225,Praia de Ipanema,Desde hotéis boutique elegantes até aconchegan...
2,454065050363887938,1.038459,Praia de Ipanema,Estas lojas se concentram nas ruas Garcia D'Av...


### Pesquisa Arpoador

In [ ]:
res = client.search(
    collection_name="tur_rj_collection",
    data=embedding_fn.encode_queries(["belezas naturais"]), #
    output_fields=["text", "subject"],
    limit=3,  # Return top 3 results
    filter='subject like "%Arpoador%"'
)

busca = []
respostas = []

for item in res[0]:
  respostas.append(item)

for resp in respostas:
    busca.append(Resposta(resp['id'],
                          resp['distance'],
                          resp['entity']['subject'],
                          resp['entity']['text']).__dict__)

df = pd.DataFrame(busca)
df

,id,distance,subject,text
0,454065051922596222,1.387758,Arpoador,"Além de ponto de recreação e lazer, o parque t..."
1,454065051922596217,1.494972,Arpoador,Localiza-se entre as praias do Diabo e de Ipanema
2,454065051922596216,1.549181,Arpoador,"Arpoador é uma praia e uma formação rochosa, a..."


### Pesquisa Praça Mauá

In [ ]:
res = client.search(
    collection_name="tur_rj_collection",
    data=embedding_fn.encode_queries(["museu"]), #
    output_fields=["text", "subject"],
    limit=3,  # Return top 3 results
    filter='subject like "%Praça Mauá%"'
)

busca = []
respostas = []

for item in res[0]:
  respostas.append(item)

for resp in respostas:
    busca.append(Resposta(resp['id'],
                          resp['distance'],
                          resp['entity']['subject'],
                          resp['entity']['text']).__dict__)

df = pd.DataFrame(busca)
df

,id,distance,subject,text
0,454065057614266782,1.085721,Praça Mauá,Projetado pelo arquiteto espanhol Santiago Cal...
1,454065057614266778,1.151405,Praça Mauá,"Como parte do sistema, foi inaugurada também a..."
2,454065057614266781,1.218160,Praça Mauá,"O edifício, de 33 andares, conta com subsolo, ..."


### Pesquisa Barra da Tijuca

In [ ]:
res = client.search(
    collection_name="tur_rj_collection",
    data=embedding_fn.encode_queries(["estrutura e arquitetura"]), #
    output_fields=["text", "subject"],
    limit=3,  # Return top 3 results
    filter='subject like "%Barra da Tijuca%"'
)

busca = []
respostas = []

for item in res[0]:
  respostas.append(item)

for resp in respostas:
    busca.append(Resposta(resp['id'],
                          resp['distance'],
                          resp['entity']['subject'],
                          resp['entity']['text']).__dict__)

df = pd.DataFrame(busca)
df

,id,distance,subject,text
0,454065072542056925,0.869040,Praia da Barra da Tijuca,A orla da praia é cortada por um vasto calçadã...
1,454065072542056914,0.886801,Praia da Barra da Tijuca,"A partir da Avenida Ayrton Senna, a praia não ..."
2,454065072542056919,0.897608,Praia da Barra da Tijuca,"Em sua orla, é muito comum encontrar pessoas p..."
